# Gloomhaven analytics

## *Val'n'Roots* Campaign Analysis

In [50]:
import sys
print(sys.executable)
import codecs
from datetime import date, datetime
import re
import requests
import typing as t

from docmodel import DocFragment, XPath

/home/lhaze/.cache/pypoetry/virtualenvs/bga-course-rdVTX_zP-py3.10/bin/python


### Get data from the source

#### Download stats of the campaing from BGG XML API2 [`plays`](https://boardgamegeek.com/wiki/page/BGG_XML_API2) endpoint

In [21]:
url = (
    b"68747470733a2f2f626f"
    b"61726467616d65676565"
    b"6b2e636f6d2f786d6c61"
    b"7069322f706c6179733f"
    b"757365726e616d653d7a"
    b"616a63656666697a7a6c"
    b"657769636b2669643d31"
    b"3734343330266d696e64"
    b"6174653d323032322d30"
    b"312d3031"
)
response = requests.get(codecs.decode(url, 'hex').decode())
assert response.status_code == 200, "Error while gathering data: BGG endpoint"

#### Preview downloaded result

In [22]:
print("\n".join(line.strip() for line in response.text[:1147].split("\n")))

<?xml version="1.0" encoding="utf-8"?><plays username="zajceffizzlewick" userid="1376730" total="32" page="1" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">
<play id="68632828" date="2023-02-02" quantity="1" length="150" incomplete="0" nowinstats="0" location="Kawadzieścia">
<item name="Gloomhaven" objecttype="thing" objectid="174430">
<subtypes>
<subtype value="boardgame" />
<subtype value="boardgameintegration" />
</subtypes>
</item>
<comments>Scenario #30 - Shrine of the Depths</comments>
<players>
<player username="zajceffizzlewick" userid="1376730" name="Ł H" startposition="6" color="Beast Tyrant" score="" new="0" rating="0" win="1" />
<player username="" userid="0" name="bartosz b." startposition="7" color="Nightshroud" score="" new="0" rating="0" win="1" />
<player username="" userid="0" name="bartek k." startposition="6" color="Soothsinger" score="" new="0" rating="0" win="1" />
<player username="" userid="0" name="kamil ch." startposition="8" color="Doomstalker" sco

### Parse data from the source

#### Build the model for the endpoint

In [59]:
class Player(DocFragment):
    name: str = XPath(".//@name")
    character: str = XPath(".//@color")
    level: t.Optional[int] = XPath(".//@startposition", clean=lambda _, s: int(v) if (v := s.get()) else None)
    won: bool = XPath(".//@win", clean=lambda _, s: s.get() == "1")

    @name.clean
    def _(self, selector_list):
        """Annotate players with identifiers."""
        value = selector_list.get()
        return {
            "Ł H": "ŁH",
            "bartek k.": "BK",
            "bartosz b.": "BB",
            "kamil ch.": "KC",
        }[value]

class Play(DocFragment):
    player: Player = XPath(".//player", model=Player, many=True)
    date: date = XPath(".//@date", clean=lambda _, s: datetime.strptime(s.get(), "%Y-%m-%d").date())
    scenario: str = XPath(".//comments/text()")
    time_length: str = XPath(".//@length")

    @scenario.clean
    def _(self, selector_list):
        value = selector_list.get()
        match = re.search(r" [a-zA-Z]*#[0-9a-zA-Z]+ ", value)
        return match.group(0).strip() if match else None

    @time_length.clean
    def _(self, selector_list):
        value = int(selector_list.get())
        hours = value // 60
        mins = value - hours * 60
        return (
            f"{hours} hours {mins} mins" if hours and mins
            else f"{hours} hours" if hours
            else f"{mins} mins"
        )

class PlayEndpoint(DocFragment):
    plays: Play = XPath("//play", model=Play, many=True)


#### Parse against the downloaded document

In [60]:
PlayEndpoint(response.text).to_dict()

{'plays': [{'player': [{'name': 'ŁH',
     'character': 'Beast Tyrant',
     'level': 6,
     'won': True},
    {'name': 'BB', 'character': 'Nightshroud', 'level': 7, 'won': True},
    {'name': 'BK', 'character': 'Soothsinger', 'level': 6, 'won': True},
    {'name': 'KC', 'character': 'Doomstalker', 'level': 8, 'won': True}],
   'date': datetime.date(2023, 2, 2),
   'scenario': '#30',
   'time_length': '2 hours 30 mins'},
  {'player': [{'name': 'ŁH',
     'character': 'Beast Tyrant',
     'level': 7,
     'won': True},
    {'name': 'BB', 'character': 'Nightshroud', 'level': 8, 'won': True},
    {'name': 'BK', 'character': 'Soothsinger', 'level': 7, 'won': True}],
   'date': datetime.date(2023, 2, 2),
   'scenario': '#25',
   'time_length': '2 hours 30 mins'},
  {'player': [{'name': 'ŁH',
     'character': 'Beast Tyrant',
     'level': 6,
     'won': False},
    {'name': 'BB', 'character': 'Nightshroud', 'level': 7, 'won': False},
    {'name': 'KC', 'character': 'Doomstalker', 'level': 